In [1]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_file, show, ColumnDataSource, save
from bokeh.palettes import viridis, brewer, magma, d3, cividis
from bokeh.models import HoverTool, LinearColorMapper
from bokeh.models.tickers import FixedTicker
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.io import output_notebook, reset_output, output_file, save

output_notebook()

Loading BokehJS ...

## Read in and merge data

In [2]:
df = pd.read_excel('../../data/raw/RVMS_Current_Property_and_BIZ_Owner_List - vCurrent (1).xlsx',
                  sheet_name = 'Biz & Prop Owner MAIN list')

naics = pd.read_excel('../../data/raw/2-6 digit_2017_Codes.xlsx')

In [3]:
df['NAICS Code'] = df['NAICS Code'].astype(object)

In [4]:
df = df.merge(naics, left_on='NAICS Code', right_on='2017 NAICS US   Code', how='inner')

In [5]:
df = df[pd.notnull(df['NAICS Code'])]

In [6]:
# Remove spaces from column names
df.columns = [c.replace(' ', '_') for c in df.columns]

In [7]:
df['NAICS_2_digit'] = df['NAICS_Code'].astype(str).str[:2]

## Generate fake data for viz purposes

In [8]:
n_rows = df.shape[0]
p = 0.25

cols = ['R2B_email_sponsorship_promotion', 'R2B_provide_resources', 'R2B_liason', 'B2R_event_participation',
       'B2R_sponsorship_donation', 'B2R_share_business_information', 'B2R_volunteer', 'B2R_use_RVMS_resources']

for col in cols:
    df[col] = np.random.binomial(n=1, p=p, size=n_rows)
    
df['R2B_score'] = df.filter(like='R2B').apply('sum',axis=1)
df['B2R_score'] = df.filter(like='B2R').apply('sum',axis=1)

low = 0.25
high = 0.75
df['R2B_score_noise'] = df['R2B_score'] + np.random.uniform(low=low, high=high, size=n_rows)
df['B2R_score_noise'] = df['B2R_score'] + np.random.uniform(low=low, high=high, size=n_rows)

## Visualization

In [ ]:
def make_dataset(df, *, noise_low=0.25, noise_high=0.75):
    n_rows = df.shape[0]
    
    # Calculate score
    df['R2B_score'] = df.filter(like='R2B').apply('sum',axis=1)
    df['B2R_score'] = df.filter(like='B2R').apply('sum',axis=1)
    df['R2B_score_noise'] = df['R2B_score'] + np.random.uniform(low=noise_low, high=noise_high, size=n_rows)
    df['B2R_score_noise'] = df['B2R_score'] + np.random.uniform(low=noise_low, high=noise_high, size=n_rows)
    
    # Add color
    cate = df['NAICS_Code'].astype(str).str[:2]
    colors = viridis(len(cats.unique()))
    colormap = {val: colors[i] for i, val in enumerate(cats.unique())}
    df['color'] = [colormap[x] for x in cats]

In [11]:
# Colors
cats = df['NAICS_Code'].astype(str).str[:2]
colors = viridis(len(cats.unique()))
colormap = {val: colors[i] for i, val in enumerate(cats.unique())}
df['color'] = [colormap[x] for x in cats]

# Radius
df['radius'] = 0.05

# Set up data source
source = ColumnDataSource(df)

# Hover tool
TOOLTIPS = [
    ("index", "$index"),
    ("Business", "@Business_Name"),
    ('Business to RVMS Score', "@B2R_score"),
    ('RVMS to Business Score', "@R2B_score"),
    ("Industry", "@2017_NAICS_US_Title"),
    ("NAICS Code", "@NAICS_Code"),
]

p = figure(title="Business Engagement Matrix", x_axis_label='Business to RVMS', y_axis_label='RVMS to Business',
          x_range=(0, 6), y_range=(0, 4), tooltips=TOOLTIPS)

# Ticks
p.xaxis.ticker = FixedTicker(ticks=[1, 2, 3, 4, 5])
p.yaxis.ticker = FixedTicker(ticks=[1, 2, 3, 4])

# Grid lines
p.xgrid.ticker = FixedTicker(ticks=[1,2,3,4,5])
p.ygrid.ticker = FixedTicker(ticks=[1,2,3,4])

p.scatter(x='B2R_score_noise', y='R2B_score_noise', fill_color='color', radius='radius', 
          source=source)

bokeh.models.renderers.GlyphRenderer(
    id='ef9830da-a03d-452c-96d8-ca3cd8467c9d',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='efda2d31-77f1-450d-bbe5-4f9c77dd88c2',
        callback=None,
        data={'Seq._No.': {'__ndarray__': 'AAAAAADgnUAAAAAAAOCdQAAAAAAA4J1AAAAAAADgnUAAAAAAAOCdQAAAAAAA4J1AAAAAAADgnUAAAAAAAOCdQAAAAAAA4J1AAAAAAADgnUAAAAAAAOCdQAAAAAAA4J1AAAAAAAAEn0AAAAAAAASfQAAAAAAABJ9AAAAAAAAEn0AAAAAAAASfQAAAAAAAiJhAAAAAAACImEAAAAAAAIiYQAAAAAAATJZAAAAAAABwmUAAAAAAAHCZQAAAAAAAHKBAAAAAAAAcoEAAAAAAABygQAAAAAAAHKBAAAAAAAAcoEAAAAAAABygQAAAAAAAHKBAAAAAAAAcoEAAAAAAABygQAAAAAAAHKBAAAAAAAAcoEAAAAAAABygQAAAAAAAHKBAAAAAAAAcoEAAAAAAABygQAAAAAAAHKBAAAAAAAAcoEAAAAAAABygQAAAAAAAEJdAAAAAAAAQl0AAAAAAABCXQAAAAAAAEJdAAAAAAAAQl0AAAAAAALCfQAAAAAAAsJ9AAAAAAACwn0AAAAAAALCfQAAAAAAAsJ9AAAAAAACwn0AAAAAAALCfQAAAAAAAEqBAAAAAAAAwk0AAAAAAADCTQAAAAAAAMJNAAAAAAACQn0AAAAAAAJCfQAAAAAAAkJ9AAAAAAACQn0AAAAAAAJCfQAAAAAAAkJ9AAAAAAACQn0AAAAAAAJCfQAAAAAAAkJ9AAAAAAACQn0AAAAAAAJCfQAAAAAAAkJ9AAAAAAAAYkkAAAAAAABiSQAAAAAAAGJJAAAAAAAAMkkAAAAAAAPCcQAAAAAAABJ1AAAAAAAAEnUAAAAAAAASdQAAAAAAATJtAAAAAAABMm0AAAAAAAEybQAAAAAAAOJlAAAAAAAA4mUAAAAAAADiZQAAAAAAAOJlAAAAAAAA4mUAAAAAAAMScQAAAAAAAxJxAAAAAAAAokkAAAAAAAOyRQAAAAAAA4JxAAAAAAACMkUAAAAAAAIyRQAAAAAAAjJFAAAAAAAAgnkAAAAAAAAyTQAAAAAAABJhAAAAAAAAEmEAAAAAAAASYQAAAAAAAsHRAAAAAAADYkUAAAAAAANiRQAAAAAAAPJNAAAAAAAA8k0AAAAAAAB6gQAAAAAAAHqBAAAAAAAAeoEAAAAAAAKySQAAAAAAAnJ9AAAAAAACcn0AAAAAAAJyfQAAAAAAAnJ9AAAAAAAAIoUAAAAAAAICVQAAAAAAAlJtAAAAAAAD0kUAAAAAAAPicQAAAAAAA+JxAAAAAAAAGoEAAAAAAAHSSQAAAAAAA4JFAAAAAAADgkUAAAAAAAOCRQAAAAAAA4JFAAAAAAAAck0AAAAAAALSRQAAAAAAAZJNAAAAAAAAYnUAAAAAAAFSSQAAAAAAA3KBAAAAAAABkmEAAAAAAAASTQAAAAAAAOKBAAAAAAAA4oEAAAAAAAJSfQAAAAAAAlJ9AAAAAAACUn0AAAAAAALCMQAAAAAAAhJ9AAAAAAACEn0AAAAAAAISfQAAAAAAASJ1AAAAAAADonEAAAAAAAOCgQAAAAAAA4KBAAAAAAADInEAAAAAAANScQAAAAAAA1JxAAAAAAADclkAAAAAAAJCaQAAAAAAAYGpAAAAAAABgk0AAAAAAADCSQAAAAAAADJxAAAAAAABYmUAAAAAAAFiZQAAAAAAAEHBAAAAAAABgkUAAAAAAAGCRQA==',
          'shape': (158,),
          'dtype': 'float64'},
         'Unnamed:_4': ['NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
          'NaN',
        

In [12]:
show(p)

In [ ]:
output_file("../../data/output/matrix.html")
save(p)
reset_output()